# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
M. Kürster  ->  M. Kürster  |  ['M. Kürster']
T. Henning  ->  T. Henning  |  ['T. Henning']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
J. Shi  ->  J. Shi  |  ['J. Shi']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
Arxiv has 52 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.11156


extracting tarball to tmp_2407.11156... done.
Retrieving document from  https://arxiv.org/e-print/2407.11289


extracting tarball to tmp_2407.11289... done.
Retrieving document from  https://arxiv.org/e-print/2407.11520
extracting tarball to tmp_2407.11520...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2142/1211882699.py:51: LatexWarning: 2407.11289 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


 done.
Retrieving document from  https://arxiv.org/e-print/2407.11557


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2142/1211882699.py:51: LatexWarning: 2407.11520 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2407.11557... done.
Retrieving document from  https://arxiv.org/e-print/2407.11572
extracting tarball to tmp_2407.11572... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.11156-b31b1b.svg)](https://arxiv.org/abs/2407.11156) | **Magnetic reconnection: an alternative explanation of radio emission in galaxy clusters**  |
|| <mark>S. Ghosh</mark>, P. Bhat |
|*Appeared on*| *2024-07-17*|
|*Comments*| *9 pages, 3 figures, comments are welcome*|
|**Abstract**|            Observations of galaxy clusters show radio emission extended over almost the system scale, necessitating mechanisms for particle acceleration. Previous models for acceleration such as diffusive shock acceleration and that due to turbulence fall short in terms of efficiency. In this letter, we propose the possibility of acceleration via magnetic reconnection. In particular, we invoke the plasmoid instability which has been previously applied to understand particle energization in high energy systems. Turbulence in galaxy clusters lead to fluctuation dynamos that are known to generate magnetic fields structures consisting of sharp reversals. These form natural sites of reconnection. We perform Particle-In-Cell (PIC) simulations of the plasmoid instability in collisionless and nonrelativistic plasmas. We show that the resulting particle energy spectra have power law indices that are consistent with that inferred from observations. Our estimates of the maximum achievable Lorentz factor is about $10^5$ indicating that acceleration due magnetic reconnection is a promising avenue for understanding the origin of nonthermal emission in galaxy clusters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.11557-b31b1b.svg)](https://arxiv.org/abs/2407.11557) | **The impact of lossy data compression on the power spectrum of the high redshift 21-cm signal with LOFAR**  |
|| J. K. Chege, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-07-17*|
|*Comments*| *12 pages, 9 figures, and 3 tables; submitted to Astronomy and Astrophysics (A&A)*|
|**Abstract**|            Current radio interferometers output multi-petabyte-scale volumes of data per year making the storage, transfer, and processing of this data a sizeable challenge. This challenge is expected to grow with the next-generation telescopes such as the Square Kilometre Array. Lossy compression of interferometric data post-correlation can abate this challenge. However, since high-redshift 21-cm studies impose strict precision requirements, the impact of such lossy data compression on the 21-cm signal power spectrum statistic should be understood. We apply Dysco visibility compression, a technique to normalize and quantize specifically designed for radio interferometric data. We establish the level of the compression noise in the power spectrum in comparison to the thermal noise as well as its coherency behavior. Finally, for optimal compression results, we compare the compression noise obtained from different compression settings to a nominal 21-cm signal power. From a single night of observation, we find that the noise introduced due to the compression is more than five orders of magnitude lower than the thermal noise level in the power spectrum. The noise does not affect calibration. The compression noise shows no correlation with the sky signal and has no measurable coherent component. The level of compression error in the power spectrum ultimately depends on the compression settings. Dysco visibility compression is found to be of insignificant concern for 21-cm power spectrum studies. Hence, data volumes can be safely reduced by factors of $\sim 4$ and with insignificant bias to the final power spectrum. Data from SKA-low will likely be compressible by the same factor as LOFAR, owing to the similarities of the two instruments. The same technique can be used to compress data from other telescopes, but a small adjustment of the compression parameters might be required.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.11572-b31b1b.svg)](https://arxiv.org/abs/2407.11572) | **Discovery of an Extremely r-process-enhanced Thin-disk Star with [Eu/H] = +0.78**  |
|| X.-J. Xie, et al. -- incl., <mark>J. Shi</mark>, <mark>R. Zhang</mark> |
|*Appeared on*| *2024-07-17*|
|*Comments*| *5 figures, 3 tables*|
|**Abstract**|            Highly r-process-enhanced stars are rare and usually metal-poor ([Fe/H] < - 1.0), and mainly populate the Milky Way halo and dwarf galaxies. This study presents the discovery of a relatively bright (V = 12.72), highly r-process-enhanced (r-II) star ([Eu/Fe] = +1.32, [Ba/Eu] = - 0.95), LAMOST J020623.21 + 494127.9. This star was selected from the Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST) medium-resolution (R ~ 7500) spectroscopic survey; follow-up high-resolution (R ~ 25,000) observations were conducted with the High Optical Resolution Spectrograph (HORuS) installed on the Gran Telescopio Canarias (GTC). The stellar parameters (${T_{\rm eff}}$ = 4130 K, $\rm log\,g $ = 1.52, $ \rm[Fe/H] $ = $ - $0.54, $\xi$ = 1.80 $ \rm{km\,{s^{-1}}} $) have been inferred taking into account non-local thermodynamic equilibrium (NLTE) effects. The abundances of [Ce/Fe], [Pr/Fe], and [Nd/Fe] are +0.19, +0.65 and +0.64, respectively, relatively low compared to the Solar r-process pattern normalized to Eu. This star has a high metallicity ([Fe/H] = - 0.54) compared to most other highly r-process-enhanced stars, and has the highest measured abundance ratio of Eu to H ([Eu/H] = +0.78). It is classified as a thin-disk star based on its kinematics, and does not appear to belong to any known stream or dwarf galaxy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.11289-b31b1b.svg)](https://arxiv.org/abs/2407.11289) | **In-lab and On-sky Closed-loop Results of Adaptive Secondary Mirrors with TNO's Hybrid Variable Reluctance Actuators**  |
|| <mark>R. Zhang</mark>, et al. |
|*Appeared on*| *2024-07-17*|
|*Comments*| *15 pages, 12 figures, SPIE Astronomical Telescopes + Instrumentation 2024*|
|**Abstract**|            We performed closed-loop lab testing of large-format deformable mirrors (DMs) with hybrid variable reluctance actuators. TNO has been developing the hybrid variable reluctance actuators in support for a new generation of adaptive secondary mirrors (ASMs), which aim to be more robust and reliable. Compared to the voice coil actuators, this new actuator technology has a higher current to force efficiency, and thus can support DMs with thicker facesheets. Before putting this new technology on-sky, it is necessary to understand how to control it and how it behaves in closed-loop. We performed closed-loop tests with the Shack-Hartmann wavefront sensor with three large-format deformable mirrors that use the TNO actuators: DM3, FLASH, and IRTF-ASM-1 ASM. The wavefront sensor and the real-time control systems were developed for the NASA Infrared Telescope Facility (IRTF) and the UH 2.2-meter telescope ASMs. We tested IRTF-ASM-1 on-sky and proved that it meets all of our performance requirements. This work presents our lab setup for the experiments, the techniques we have employed to drive these new ASMs, the results of our closed-loop lab tests for FLASH and IRTF-ASM-1, and the on-sky closed-loop results of IRTF-ASM-1 ASM.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.11520-b31b1b.svg)](https://arxiv.org/abs/2407.11520) | **The CARMENES search for exoplanets around M dwarfs. Revisiting the GJ 581 multi-planetary system with new Doppler measurements from CARMENES, HARPS, and HIRES**  |
|| A. v. S. T. Trifonov, et al. -- incl., <mark>M. Kürster</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-07-17*|
|*Comments*| *21 pages, 8 Figures, 5 Tables, 1 Appendix, Accepted for publication in A&A (20th March 2024), Data available at the CDS*|
|**Abstract**|            GJ 581 is a nearby M dwarf known to host a packed multiple planet system with 2 super-Earths and a Neptune-mass planet. We present new orbital analyses of the system, utilizing recent RV data obtained from the CARMENES spectrograph combined with newly reprocessed archival data from the HARPS and HIRES spectrographs. Our aim was to analyze the post-discovery spectroscopic data of GJ 581, which were obtained with CARMENES. In addition, we used publicly available HIRES and HARPS spectroscopic data to seek evidence of the known and disputed exoplanets in this system. We aimed to investigate the stellar activity of GJ 581 and update the planetary system's orbital parameters using state-of-the-art numerical models and techniques. We performed a periodogram analysis of the available precise CARMENES, HIRES, and HARPS RVs and of stellar activity indicators. We conducted detailed orbital analyses by testing various orbital configurations consistent with the RV data. We studied the posterior probability distribution of the parameters fit to the data and explored the long-term stability and overall orbital dynamics of the system. We refined the orbital parameters of the system using the most precise and complete set of Doppler data available. Consistent with the existing literature, we confirm that the system is unequivocally composed of only 3 planets detectable in the present data, dismissing the putative planet GJ 581 d as an artifact of stellar activity. Our N-body fit reveals that the system's inclination is i $=$ 47.0 deg, which implies that the planets could be up to 30% more massive than their previously reported minimum masses. Furthermore, we report that the system exhibits long-term stability, as indicated by the posterior probability distribution, characterized by secular dynamical interactions without the involvement of mean motion resonances.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

137  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
